# POSTTEST4

><p> Nama : Tito Darmawan <br> NIM : 2109106042 <br> Dataset : </p>

[Goodreads-books](https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks)

## Import

In [270]:
import numpy as np
import pandas as pd

dfawal = pd.read_csv(
    "books.csv", 
    on_bad_lines="skip"
)

dfawal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


## Bagi ke dua Variabel X dan Y

In [218]:
from sklearn.model_selection import train_test_split

X = dfawal.iloc[:, : -1]
y = dfawal.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print("Pembagian Dataset : \n")
print("Dimensi X_train", X_train.shape)
print("Dimensi X_test", X_test.shape)
print("Dimensi y_train", y_train.shape)
print("Dimensi y_test", y_test.shape)

Pembagian Dataset : 

Dimensi X_train (7786, 11)
Dimensi X_test (3337, 11)
Dimensi y_train (7786,)
Dimensi y_test (3337,)


## Data Transforming <br>
### Normalisasi
> <b> Menormalisasikan jumlah review buku
      Perbandingan dapat di lihat pada "text_reviews_count" untuk sebelum dan "count_norm" untuk sesudah

In [118]:
from sklearn.preprocessing import MinMaxScaler

data_show = dfawal[["text_reviews_count"]]
data_show.columns = ["text_reviews_count"]

mms = MinMaxScaler()
arrN = mms.fit_transform(data_show[["text_reviews_count"]])
dfN = pd.DataFrame(arrN)
dfN.columns =["count_norm"]

data_show = data_show.join(dfN["count_norm"]).reset_index()
data_show = data_show.drop("index", axis=1)

print("Perbandingan sebelum dan sesudah normalisasi : ")
data_show.head()

Perbandingan sebelum dan sesudah normalisasi : 


,text_reviews_count,count_norm
0,27591,0.292696
1,29221,0.309988
2,244,0.002588
3,36325,0.385350
4,164,0.001740


### Standarisasi
> <b> Menstandarisasikan semua atribut numeric,
      Perbandingan dapat di lihat di bawah

In [181]:
from sklearn.preprocessing import StandardScaler

data_show2 = dfawal.select_dtypes(include='number')

ss = StandardScaler()
arrS = ss.fit_transform(data_show2)
dfS = pd.DataFrame(arrS)
dfS.columns = ["bookID_stand", "average_rating_stand", "isbn13_stand", "num_pages_stand", "ratings_count_stand", "text_reviews_coun_stand"]

print("Perbandingan standar devirasi sebelum dan sesudah : ")
np.std(data_show2)

Perbandingan standar devirasi sebelum dan sesudah : 


bookID                1.309414e+04
average_rating        3.504696e-01
isbn13                4.429559e+11
  num_pages           2.411418e+02
ratings_count         1.124941e+05
text_reviews_count    2.576504e+03
dtype: float64

In [182]:
np.std(dfS)

bookID_stand               1.0
average_rating_stand       1.0
isbn13_stand               1.0
num_pages_stand            1.0
ratings_count_stand        1.0
text_reviews_coun_stand    1.0
dtype: float64

### Data Cleaning
## Null
> <b> Karna tidak ada yang null maka di buat dengan library random <br> <br>
  <b> NOTE : Null akan betambah setiap blok kode di run karna di generate secara acak dengan kemungkinan 10% untuk null. Jika blok kode di run terus menerus       maka semua data akan null <br>

In [196]:
import random

dfrusak = dfawal.select_dtypes(include='number')

ix = [(row, col) for row in range(dfrusak.shape[0]) for col in range(dfrusak.shape[1])]
for row, col in random.sample(ix, int(round(.1*len(ix)))):
    dfrusak.iat[row, col] = np.nan
    
dfrusak.head()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
0,1.0,4.57,9.780440e+12,652.0,NaN,NaN
1,2.0,4.49,9.780439e+12,870.0,2153167.0,29221.0
2,4.0,NaN,9.780440e+12,352.0,6333.0,244.0
3,NaN,4.56,9.780440e+12,435.0,NaN,36325.0
4,8.0,4.78,NaN,2690.0,41428.0,NaN


In [197]:
dfrusak.isna().sum()

bookID                1100
average_rating        1145
isbn13                1104
  num_pages           1103
ratings_count         1146
text_reviews_count    1076
dtype: int64

## Menangani data null
### Substitusi nilai null (float dengan strategi mean)
> <b> semua value yang kosong akan di isi rata-rata record yang tidak kosong <b>

In [183]:
dfrusak.dtypes

bookID                float64
average_rating        float64
isbn13                float64
  num_pages           float64
ratings_count         float64
text_reviews_count    float64
dtype: object

In [209]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy = "mean")
arrC = si.fit_transform(dfrusak)
dfSub = pd.DataFrame(arrC)
dfSub.columns = ["bookID_stand", "average_rating_stand", "isbn13_stand", "num_pages_stand", "ratings_count_stand", "text_reviews_coun_stand"]

print("Setelah Data Cleaning : \n")
dfSub.isna().sum()

Setelah Data Cleaning : 



bookID_stand               0
average_rating_stand       0
isbn13_stand               0
num_pages_stand            0
ratings_count_stand        0
text_reviews_coun_stand    0
dtype: int64

In [210]:
dfSub.head() 

,bookID_stand,average_rating_stand,isbn13_stand,num_pages_stand,ratings_count_stand,text_reviews_coun_stand
0,1.000000,4.570000,9.780440e+12,652.0,1.758373e+04,545.162337
1,2.000000,4.490000,9.780439e+12,870.0,2.153167e+06,29221.000000
2,4.000000,3.934303,9.780440e+12,352.0,6.333000e+03,244.000000
3,21281.819715,4.560000,9.780440e+12,435.0,1.758373e+04,36325.000000
4,8.000000,4.780000,9.759449e+12,2690.0,4.142800e+04,545.162337


## Duplikat
> <b> Karna tidak ada yang duplikat maka di buat dengan mengulang setiap record

In [255]:
dfdupli = pd.concat([dfawal]*2, ignore_index=True)
dfdupli.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
22241     True
22242     True
22243     True
22244     True
22245     True
Length: 22246, dtype: bool

In [260]:
print("Jumlah duplikat : ")
dfdupli.duplicated().sum()

Jumlah duplikat : 


11123

## Menangani Data Duplikat
> <b> drop_duplicates <b>|  akan menghapus semua record yang di tambah tadi karna berisikan value yang sama

In [261]:
dfdupli.drop_duplicates(inplace = True)
dfdupli.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
11118    False
11119    False
11120    False
11121    False
11122    False
Length: 11123, dtype: bool

In [264]:
print("Jumlah duplikat : ")
dfdupli.duplicated().sum()

Jumlah duplikat : 


0

## Ubah tipe data salah satu atribut angka
> <b> Mengubah tipe data angka menjadi objek

In [273]:
print("Sebelum : ")
dfawal["isbn13"].dtypes

dtype('int64')

In [275]:
print("Sesudah : ")
dfubah = dfawal[["isbn13"]].astype("object")
dfubah["isbn13"].dtypes

Sesudah : 


dtype('O')

## One Hot Encoding
> <b> Kategori yang akan di encoding adalah "language_code" dengan 27 kategori

In [327]:
from sklearn.preprocessing import OneHotEncoder

#untuk penamaan kolom ketegori onehotencoding
data_lang = dfawal["language_code"].value_counts().rename_axis("language_code").reset_index(name="count")
czip = zip(data_lang["language_code"])
cname = [nama_kategori[0] for nama_kategori in list(czip)]

#Encoding
ohe = OneHotEncoder(sparse=False)
arr_ohe = ohe.fit_transform(dfawal[["language_code"]])

dfH = pd.DataFrame(arr_ohe)
dfH.columns = cname
dfH = dfawal.join(dfH)

dfH.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,...,srp,nl,msa,glg,wel,ara,nor,tur,gla,ale
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
